<a href="https://colab.research.google.com/github/Rutul-Bokade/Portfolio/blob/main/route_optimisation_ss_good.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

geocoding using os api nad matricing it to increase request numbers

In [ ]:
# ============================================================
# ⚡ ASYNC OPENROUTESERVICE GEOCODING + DISTANCE BUCKETS
# ============================================================

!pip install pandas openpyxl aiohttp nest_asyncio openrouteservice tqdm geopy

import pandas as pd
import aiohttp
import asyncio
from tqdm import tqdm
import nest_asyncio
import openrouteservice
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import numpy as np

nest_asyncio.apply()

# =======================
# 🔑 ORS API KEY
# =======================
ORS_API_KEY = "eyJvcmciOiI1YjNjZTM1OTc4NTExMTAwMDFjZjYyNDgiLCJpZCI6ImRkMzY1ZGQ0YzU2ZTRmYjg5NTc2MzdmOGQ1ZTE4YTg2IiwiaCI6Im11cm11cjY0In0="
client = openrouteservice.Client(key=ORS_API_KEY)

# =======================
# 📍 ORIGIN ADDRESS
# =======================
ORIGIN_ADDRESS = "5251 California Ave, Irvine, CA 92617"

# Use Nominatim to geocode origin
geolocator = Nominatim(user_agent="ors_async_demo")
_geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1, swallow_exceptions=True)

origin = _geocode(ORIGIN_ADDRESS)
if origin:
    origin_coords = (origin.longitude, origin.latitude)
    print(f"✅ Origin: {origin_coords}")
else:
    raise ValueError("❌ Could not geocode the origin address.")

# =======================
# 📂 LOAD EXCEL
# =======================
file_path = "/content/Sales Strategy - 2-way path.xlsx"
df = pd.read_excel(file_path)
df.columns = df.columns.str.strip()

address_col = [c for c in df.columns if 'address' in c.lower()][0]
company_col = [c for c in df.columns if 'company' in c.lower()][0]
print(f"Using columns: {company_col} | {address_col}")

# ============================================================
# 🧠 ASYNC GEOCODE FUNCTION
# ============================================================
async def ors_geocode_async(session, address):
    """Asynchronously geocode using ORS Pelias search."""
    if not address or not str(address).strip():
        return None, None

    url = "https://api.openrouteservice.org/geocode/search"
    params = {"api_key": ORS_API_KEY, "text": address}

    try:
        async with session.get(url, params=params) as response:
            data = await response.json()
            if "features" in data and data["features"]:
                coords = data["features"][0]["geometry"]["coordinates"]
                lon, lat = coords
                return lat, lon
    except Exception:
        return None, None
    return None, None

# ============================================================
# 📏 ASYNC DISTANCE FUNCTION
# ============================================================
async def ors_distance_async(session, origin_coords, dest_coords):
    """Asynchronously get driving distance in miles."""
    if not dest_coords or None in dest_coords:
        return None

    url = "https://api.openrouteservice.org/v2/directions/driving-car"
    headers = {"Authorization": ORS_API_KEY, "Content-Type": "application/json"}
    body = {"coordinates": [origin_coords, dest_coords]}

    try:
        async with session.post(url, headers=headers, json=body) as resp:
            data = await resp.json()
            if "features" in data and data["features"]:
                dist_m = data["features"][0]["properties"]["summary"]["distance"]
                return dist_m / 1609.34  # convert meters to miles
    except Exception:
        return None

# ============================================================
# ⚙️ MATRIXED EXECUTION
# ============================================================
async def process_all(addresses, batch_size=25):
    latitudes, longitudes, distances = [], [], []

    async with aiohttp.ClientSession() as session:
        for i in tqdm(range(0, len(addresses), batch_size), desc="Matrixed Geocoding + Distance"):
            batch = addresses[i:i + batch_size]
            # 1️⃣ Geocode all addresses in batch
            geocode_tasks = [ors_geocode_async(session, addr) for addr in batch]
            results = await asyncio.gather(*geocode_tasks)

            # 2️⃣ Compute distances concurrently
            distance_tasks = []
            for lat, lon in results:
                if lat and lon:
                    distance_tasks.append(ors_distance_async(session, origin_coords, (lon, lat)))
                else:
                    distance_tasks.append(asyncio.sleep(0, result=None))  # dummy
            dists = await asyncio.gather(*distance_tasks)

            # 3️⃣ Save batch results
            for (lat, lon), dist in zip(results, dists):
                latitudes.append(lat)
                longitudes.append(lon)
                distances.append(dist)

            await asyncio.sleep(1.0)  # pause between batches to respect rate limits

    return latitudes, longitudes, distances

# ============================================================
# 🚀 RUN MATRIXED TASK
# ============================================================
addresses = df[address_col].fillna("").tolist()
latitudes, longitudes, distances = asyncio.run(process_all(addresses, batch_size=25))

df["Latitude"] = latitudes
df["Longitude"] = longitudes
df["Distance_from_Origin_miles"] = distances

# ============================================================
# 🧭 LABEL DISTANCE RANGES + ESTIMATED DRIVE TIMES
# ============================================================
df["Distance_from_Origin_miles"] = df["Distance_from_Origin_miles"].fillna(0)

# Define bins and labels
bins = [0, 5, 10, 15, 20, 25, 30, 40, 50, np.inf]
labels = ["0-5", "5-10", "10-15", "15-20", "20-25", "25-30", "30-40", "40-50", "50+"]

# Distance bucket column
df["Distance_Range_miles"] = pd.cut(df["Distance_from_Origin_miles"], bins=bins, labels=labels, right=False)

# Time mapping
time_map = {
    "0-5": "5 mins",
    "5-10": "10 mins",
    "10-15": "15 mins",
    "15-20": "20 mins",
    "20-25": "25 mins",
    "25-30": "30 mins",
    "30-40": "40 mins",
    "40-50": "50 mins",
    "50+": "60+ mins"
}

df["Estimated_Drive_Time"] = df["Distance_Range_miles"].map(time_map)

# ============================================================
# 💾 SAVE OUTPUT
# ============================================================
output_path_labeled = "/content/ORS_Geocoded_Labeled_Distances.xlsx"
df[[company_col, address_col, "Latitude", "Longitude",
    "Distance_from_Origin_miles", "Distance_Range_miles", "Estimated_Drive_Time"]].to_excel(output_path_labeled, index=False)

print(f"✅ All done! File saved to: {output_path_labeled}")


✅ Origin: (-117.8513473, 33.6381621)
Using columns: Company | Address


Matrixed Geocoding + Distance: 100%|██████████| 5/5 [00:31<00:00,  6.24s/it]

✅ All done! File saved to: /content/ORS_Geocoded_Labeled_Distances.xlsx



/tmp/ipython-input-1247969244.py:140: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["Distance_from_Origin_miles"] = df["Distance_from_Origin_miles"].fillna(0)


route optimisation plus folium maping

In [ ]:
print(df.columns.tolist())

['Company', 'Address', 'Latitude', 'Longitude', 'Distance_from_Origin_miles', 'Distance_Range_miles', 'Estimated_Drive_Time']


In [ ]:
# ============================================================
# 🚗 MULTI-REP OPTIMIZED ROUTE PLANNER (NO API)
# Start all reps at origin, constraints: work hours, buffer, target locations
# ============================================================

!pip install folium pandas numpy geopy openpyxl tqdm > /dev/null

import pandas as pd
import numpy as np
from geopy.distance import geodesic
import folium
from folium import plugins
from itertools import cycle
from tqdm import tqdm

# ------------------------
# ⚙️ USER INPUTS
# ------------------------
EXCEL_PATH = "/content/ORS_Geocoded_Labeled_Distances.xlsx"  # your pre-geocoded file
ORIGIN = (33.6533, -117.81)  # 5251 California Ave
NUM_REPS = int(input("Enter number of sales reps: "))
WORK_HOURS = float(input("Enter available work hours per rep (e.g., 8): "))
AVG_SPEED = float(input("Enter average driving speed (mph): "))
BUFFER_TIME = float(input("Enter buffer time per stop (minutes): "))
TARGET_LOCATIONS = int(input("Enter target number of locations per rep: "))

# ------------------------
# 🧾 LOAD DATA
# ------------------------
df = pd.read_excel(EXCEL_PATH)
lat_col = "Latitude"
lon_col = "Longitude"
name_col = "Company"
time_col = "Estimated_Drive_Time"  # minutes

locations = df[[name_col, lat_col, lon_col, time_col]].dropna().reset_index(drop=True)
print(f"✅ Loaded {len(locations)} locations.")

# ------------------------
# 🧮 HELPER FUNCTIONS
# ------------------------
def travel_time_mins(point1, point2, avg_speed):
    """Compute travel time in minutes using geodesic distance."""
    dist_miles = geodesic(point1, point2).miles
    return (dist_miles / avg_speed) * 60

def mins_to_clock(minutes):
    hours = int(minutes // 60)
    mins = int(minutes % 60)
    return f"{hours:02d}:{mins:02d}"

# ------------------------
# 🧭 ROUTE SIMULATION FUNCTION
# ------------------------
def simulate_routes():
    routes = []
    unvisited = locations.copy().reset_index(drop=True)
    max_minutes = WORK_HOURS * 60
    rep_status = {rep: {"completed": False} for rep in range(1, NUM_REPS+1)}

    for rep_id in range(1, NUM_REPS+1):
        current_loc = ORIGIN
        total_time = 0
        visited_count = 0
        rep_visits = []

        while visited_count < TARGET_LOCATIONS and not unvisited.empty:
            # Compute travel time from current location to all remaining locations
            unvisited["travel_time"] = unvisited.apply(
                lambda row: travel_time_mins(current_loc, (row[lat_col], row[lon_col]), AVG_SPEED), axis=1
            )
            # Select nearest feasible location
            feasible = unvisited[unvisited["travel_time"] + total_time + BUFFER_TIME <= max_minutes]

            if feasible.empty:
                break  # can't visit more within time

            next_idx = feasible["travel_time"].idxmin()
            next_row = unvisited.loc[next_idx]

            # Add to route
            total_time += next_row["travel_time"] + BUFFER_TIME
            rep_visits.append({
                "rep": rep_id,
                "company": next_row[name_col],
                "lat": next_row[lat_col],
                "lon": next_row[lon_col],
                "travel_time_min": round(next_row["travel_time"],2),
                "total_time_min": round(total_time,2),
                "time_on_clock": mins_to_clock(total_time)
            })
            visited_count += 1
            current_loc = (next_row[lat_col], next_row[lon_col])
            unvisited.drop(next_idx, inplace=True)
            unvisited = unvisited.reset_index(drop=True)

        # Decide if rep returns to origin
        travel_back = travel_time_mins(current_loc, ORIGIN, AVG_SPEED)
        returned = False
        if total_time + travel_back <= max_minutes:
            total_time += travel_back
            returned = True
        for r in rep_visits:
            r["returned_to_origin"] = returned

        # Mark completion status
        rep_status[rep_id]["completed"] = visited_count >= TARGET_LOCATIONS

        routes.extend(rep_visits)
        print(f"Rep {rep_id} visited {visited_count} locations. Returned to origin: {returned}. Completed target: {rep_status[rep_id]['completed']}")

    return pd.DataFrame(routes), rep_status

# ------------------------
# 🧾 RUN SIMULATION
# ------------------------
tqdm.pandas()
visits_df, rep_status = simulate_routes()

# ------------------------
# 📝 EXPORT EXCEL
# ------------------------
visits_df[["rep","company","time_on_clock"]].to_excel("/content/Rep_Visit_Times.xlsx", index=False)
print("✅ Excel saved: Rep_Visit_Times.xlsx")

# ------------------------
# 🗺️ FOLIUM MAP
# ------------------------
m = folium.Map(location=ORIGIN, zoom_start=10)
folium.Marker(ORIGIN, icon=folium.Icon(color="black", icon="home"), popup="Origin").add_to(m)
colors = cycle(["red","blue","green","purple","orange","darkred","cadetblue"])

for rep_id, group in visits_df.groupby("rep"):
    color = next(colors)
    coords = [(ORIGIN[0], ORIGIN[1])] + list(zip(group["lat"], group["lon"]))
    folium.PolyLine(coords, color=color, weight=3, opacity=0.8, tooltip=f"Rep {rep_id} Route").add_to(m)
    for _, r in group.iterrows():
        folium.CircleMarker(
            location=(r["lat"], r["lon"]),
            radius=6,
            color=color,
            fill=True,
            fill_opacity=0.8,
            popup=f"Rep {rep_id} - {r['company']}\nTime: {r['time_on_clock']}\nReturned: {r['returned_to_origin']}",
        ).add_to(m)

# Add tools
plugins.Fullscreen().add_to(m)
plugins.MeasureControl().add_to(m)

# ------------------------
# ✅ REPORT COMPLETION
# ------------------------
completed = sum([v["completed"] for v in rep_status.values()])
not_completed = NUM_REPS - completed
print(f"\nSummary: {completed} reps completed their target, {not_completed} reps did not.")

# Display map in Colab
m


Enter number of sales reps: 5
Enter available work hours per rep (e.g., 8): 8
Enter average driving speed (mph): 35
Enter buffer time per stop (minutes): 30
Enter target number of locations per rep: 7
✅ Loaded 90 locations.
Rep 1 visited 7 locations. Returned to origin: True. Completed target: True
Rep 2 visited 7 locations. Returned to origin: True. Completed target: True
Rep 3 visited 7 locations. Returned to origin: True. Completed target: True
Rep 4 visited 7 locations. Returned to origin: True. Completed target: True
Rep 5 visited 7 locations. Returned to origin: True. Completed target: True
✅ Excel saved: Rep_Visit_Times.xlsx

Summary: 5 reps completed their target, 0 reps did not.


assigned sheet

In [ ]:
import pandas as pd

# --- File Paths ---
ors_file = "ORS_Geocoded_Labeled_Distances_corrected.xlsx"
routes_file = "Simulated_Routes.xlsx"
output_file = "ORS_Merged_With_Routes_Cleaned.xlsx"

# --- Load Data ---
ors_df = pd.read_excel(ors_file)
routes_df = pd.read_excel(routes_file)

# --- Normalize company names for consistent matching ---
ors_df["Company_norm"] = ors_df["Company"].str.strip().str.lower()
routes_df["company_norm"] = routes_df["company"].str.strip().str.lower()

# --- Merge route info onto ORS ---
merged = ors_df.merge(
    routes_df[["rep", "company_norm", "lat", "lon", "travel_time_min", "total_time_min"]],
    how="left",
    left_on="Company_norm",
    right_on="company_norm"
)

# --- Add Visited / NVY column ---
merged["Visited"] = merged["rep"].apply(lambda x: "Visited" if pd.notna(x) else "NVY")

# --- Rename rep column ---
merged.rename(columns={"rep": "RepNumber"}, inplace=True)

# --- Drop unwanted columns ---
cols_to_drop = [
    "Distance_from_Origin_miles",
    "Distance_Range_miles",
    "Estimated_Drive_Time",
    "Geocode_Missing",
    "lat",
    "lon",
    "Distance_from_Origin_miles_corrected"  # if exists
]

merged.drop(columns=[c for c in cols_to_drop if c in merged.columns], inplace=True)

# --- Drop helper columns ---
merged.drop(columns=["Company_norm", "company_norm"], inplace=True, errors="ignore")

# --- Sort neatly ---
merged.sort_values(by=["Company", "RepNumber"], inplace=True)

# --- Export final Excel ---
merged.to_excel(output_file, index=False)

print(f"✅ Cleaned merged Excel created successfully: {output_file}")


✅ Cleaned merged Excel created successfully: ORS_Merged_With_Routes_Cleaned.xlsx


In [ ]:
import pandas as pd

# --- File paths ---
sales_file = "/content/Sales Visits - Data(Visits).csv"
merged_file = "ORS_Merged_With_Routes_Cleaned.xlsx"
output_file = "Sales_Visits_With_RepNumber.csv"

# --- Load data (handle encoding safely) ---
try:
    sales_df = pd.read_csv(sales_file, encoding='utf-8-sig')
except UnicodeDecodeError:
    sales_df = pd.read_csv(sales_file, encoding='ISO-8859-1')

merged_df = pd.read_excel(merged_file)

# --- Normalize company names for matching ---
sales_df["Company_norm"] = sales_df["Company"].str.strip().str.lower()
merged_df["Company_norm"] = merged_df["Company"].str.strip().str.lower()

# --- Merge to add rep numbers ---
final_df = sales_df.merge(
    merged_df[["Company_norm", "RepNumber"]],
    on="Company_norm",
    how="left"
)

# --- Fill NVY where no rep found ---
final_df["RepNumber"] = final_df["RepNumber"].fillna("NVY")

# --- Drop helper column ---
final_df.drop(columns=["Company_norm"], inplace=True)

# --- Save output ---
final_df.to_csv(output_file, index=False, encoding='utf-8-sig')

print(f"✅ File created successfully: {output_file}")


✅ File created successfully: Sales_Visits_With_RepNumber.csv
